In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

from app.models import DatedActivityLog

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [2]:
@tool
def get_protein_intake(weight: int) -> int:
    """Returns the recommended protein intake for a person based on their weight."""
    return weight * 0.8

@tool
def get_bmr(weight: int, height: int, age: int, gender: str, activity_level: str) -> tuple:
    """
    Returns the Basal Metabolic Rate (BMR) and Total Daily Energy Expenditure (TDEE) for a person based on their weight, height, age, gender (male, female), and activity level (sedentary, lightly active, moderately active, very active, super active).

    Parameters:
        weight (float): Weight of the person in kilograms.
        height (float): Height of the person in centimeters.
        age (int): Age of the person in years.
        gender (str): Gender of the person, either 'male' or 'female'.
        activity_level (str): Activity level of the person, one of the following:
            - 'sedentary'
            - 'lightly active'
            - 'moderately active'
            - 'very active'
            - 'super active'

    Returns:
        tuple: A tuple containing:
            - BMR (float): Basal Metabolic Rate.
            - TDEE (float): Total Daily Energy Expenditure.
    """
    
    # Calculate BMR
    if gender == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    elif gender == 'female':
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    else:
        raise ValueError("Gender must be 'male' or 'female'")

    # Calculate TDEE based on activity level
    activity_multipliers = {
        'sedentary': 1.2,
        'lightly active': 1.375,
        'moderately active': 1.55,
        'very active': 1.725,
        'super active': 1.9
    }

    if activity_level not in activity_multipliers:
        raise ValueError("Activity level must be one of: 'sedentary', 'lightly active', 'moderately active', 'very active', 'super active'")

    tdee = bmr * activity_multipliers[activity_level]
    
    return bmr, tdee

    

In [3]:
tools = [get_protein_intake, get_bmr]

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [7]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
list(agent_executor.stream({"input": "I am 80kg, how much protein should I eat?"}))



> Entering new AgentExecutor chain...

Invoking: `get_protein_intake` with `{'weight': 80}`


64.0Based on your weight of 80 kg, you should aim to consume approximately 64 grams of protein per day.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_protein_intake', tool_input={'weight': 80}, log="\nInvoking: `get_protein_intake` with `{'weight': 80}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ebqt41H6CMamsrFhSddVixZ1', 'function': {'arguments': '{"weight":80}', 'name': 'get_protein_intake'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d576307f90'}, id='run-d08a9b3c-f058-489c-8297-c869923be2bf', tool_calls=[{'name': 'get_protein_intake', 'args': {'weight': 80}, 'id': 'call_ebqt41H6CMamsrFhSddVixZ1'}], tool_call_chunks=[{'name': 'get_protein_intake', 'args': '{"weight":80}', 'id': 'call_ebqt41H6CMamsrFhSddVixZ1', 'index': 0}])], tool_call_id='call_ebqt41H6CMamsrFhSddVixZ1')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ebqt41H6CMamsrFhSddVixZ1', 'function': {'arguments': '{"weight":80}

In [9]:
list(agent_executor.stream({"input": "I am 90kg, 186cm, 25 years old and male. I am moderately active, what is my BMR and TDEE?"}))



> Entering new AgentExecutor chain...

Invoking: `get_bmr` with `{'weight': 90, 'height': 186, 'age': 25, 'gender': 'male', 'activity_level': 'moderately active'}`


(1942.5, 3010.875)Your Basal Metabolic Rate (BMR) is approximately 1942.5 calories per day, and your Total Daily Energy Expenditure (TDEE) is approximately 3010.875 calories per day.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_bmr', tool_input={'weight': 90, 'height': 186, 'age': 25, 'gender': 'male', 'activity_level': 'moderately active'}, log="\nInvoking: `get_bmr` with `{'weight': 90, 'height': 186, 'age': 25, 'gender': 'male', 'activity_level': 'moderately active'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_P11gdCX9LZnrP1AZgJb0JdZ8', 'function': {'arguments': '{"weight":90,"height":186,"age":25,"gender":"male","activity_level":"moderately active"}', 'name': 'get_bmr'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ce0793330f'}, id='run-42edf764-157b-4e5a-a6f9-3257c72154a5', tool_calls=[{'name': 'get_bmr', 'args': {'weight': 90, 'height': 186, 'age': 25, 'gender': 'male', 'activity_level': 'moderately active'}, 'id': 'call_P11gdCX9LZnrP1AZgJb0JdZ8'}], tool_call_chunks=[{'name': 'get_bmr', 'args': '{"weight":90,"heig